# WASDE Data Exploration


1. Libraries and setup

In [1]:
# -- Setup environment and imports --

# Add 'src' directory to path to import project modules
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent / "src"))

# Load environment variables securely (e.g., WASDE_JWT)
from dotenv import load_dotenv
import os
load_dotenv()
my_token = os.getenv("WASDE_JWT")

# Core imports
from datetime import datetime
import pandas as pd
import os

# Project configuration and functions
from config import (
    WASDE_FOLDER,
    WHEAT,
    WHEAT_CURRENT,
    WHEAT_NEXT,
    WHEAT_OUTLOOK,
    CORN,
    CORN_CURRENT,
    CORN_NEXT,
    CORN_OUTLOOK,
    SOYBEAN,
    SOYBEAN_CURRENT,
    SOYBEAN_NEXT,
    SOYBEAN_OUTLOOK,
    SOYBEAN_OIL,
    SOYBEAN_OIL_CURRENT,
    SOYBEAN_OIL_NEXT,
    SOYBEAN_OIL_OUTLOOK,
    SOYBEAN_MEAL,
    SOYBEAN_MEAL_CURRENT,
    SOYBEAN_MEAL_NEXT,
    SOYBEAN_MEAL_OUTLOOK,
    COMMODITY
)

from wasde_functions import (
    fetch_wasde_releases, 
    download_release_files,
    process_wheat,
    process_corn,
    process_soybean,
    process_soybean_oil,
    process_soybean_meal   
)


2. Identifying reports available and download of the csv/xml files

In [2]:
# Step 1: Fetch WASDE releases metadata using your secure token
releases = fetch_wasde_releases(token=my_token)
print(f"{len(releases)} releases found.")

# Step 2: Download the XML files (limit optional for testing)
download_release_files(releases, limit=None)


317 releases found.
Downloaded: 2025-06-12_wasde0625.xls
Already exists: 2025-05-12_wasde0525v2.xls, skipping.
Already exists: 2025-04-10_wasde0425v2.xls, skipping.
Already exists: 2025-03-11_wasde0325.xls, skipping.
Already exists: 2025-02-11_wasde0225.xls, skipping.
Already exists: 2025-01-10_wasde0125.xls, skipping.
Already exists: 2024-12-10_wasde1224v2.xls, skipping.
Already exists: 2024-11-08_wasde1124.xls, skipping.
Already exists: 2024-10-11_wasde1024.xls, skipping.
Already exists: 2024-09-12_wasde0924.xls, skipping.
Already exists: 2024-08-12_wasde0824.xls, skipping.
Already exists: 2024-07-12_wasde0724.xls, skipping.
Already exists: 2024-06-12_wasde0624.xls, skipping.
Already exists: 2024-05-10_wasde0524.xls, skipping.
Already exists: 2024-04-11_wasde0424.xls, skipping.
Already exists: 2024-03-08_wasde0324.xls, skipping.
Already exists: 2024-02-08_wasde0224.xls, skipping.
Already exists: 2024-01-12_wasde0124.xls, skipping.
Already exists: 2023-12-08_wasde1223.xls, skipping.
A

3. Processing wheat

In [3]:
files = list(WASDE_FOLDER.glob('*.xls'))

latest_file_by_date = {}
for file in files:
    basename = os.path.basename(file)
    report_date = basename.split('_')[0]
    latest_file_by_date[report_date] = file

dfs_wheat = []
dfs_current = []
dfs_next = []
dfs_outlook = []

for file in latest_file_by_date.values():
    result = process_wheat(file)
    if result is None:
        continue

    df_wheat, df_current, df_next, df_outlook = result
    dfs_wheat.append(df_wheat)
    dfs_current.append(df_current)
    dfs_next.append(df_next)
    dfs_outlook.append(df_outlook)

df_wheat = pd.concat(dfs_wheat, ignore_index=True)
df_wheat_current = pd.concat(dfs_current, ignore_index=True, sort=False)
df_wheat_next = pd.concat(dfs_next, ignore_index=True, sort=False)
df_wheat_outlook = pd.concat(dfs_outlook, ignore_index=True, sort=False)

df_wheat_current = df_wheat_current.reset_index(drop=True)
df_wheat_next = df_wheat_next.reset_index(drop=True)
df_wheat_outlook = df_wheat_outlook.reset_index(drop=True)
df_wheat = df_wheat.reset_index(drop=True)

df_wheat_current.to_excel(WHEAT_CURRENT)
df_wheat_next.to_excel(WHEAT_NEXT)
df_wheat_outlook.to_excel(WHEAT_OUTLOOK)
df_wheat.to_excel(WHEAT)

print(f"✅ df_wheat created with {len(df_wheat)} lines.")
print(f"✅ df_wheat_current created with {len(df_wheat_current)} lines.")
print(f"✅ df_wheat_next created with {len(df_wheat_next)} lines.")
print(f"✅ df_wheat_outlook created with {len(df_wheat_outlook)} lines.")

⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-09_wasde-05-09-2001_rice-revision.xls: Worksheet named 'Page 18' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-10_wasde-05-10-2001_wheat-revision.xls: Worksheet named 'Page 18' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-05-10_wasde-05-10-2002_world_grains_revision.xls: Worksheet named 'Page 18' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-11-12_wasde-11-12-2002_China_wheat_revision.xls: Worksheet named 'Page 18' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2003-07-11_wasde-07-11-2003_world_oilseed_revision.xls: Worksheet named 'Page 18' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2004-05-12_wasde-05-12-2004_world_coarse_grains_revision.xls: 

4. Processing corn

In [4]:
files = list(WASDE_FOLDER.glob('*.xls'))

latest_file_by_date = {}
for file in files:
    basename = os.path.basename(file)
    report_date = basename.split('_')[0]
    latest_file_by_date[report_date] = file

dfs_corn = []
dfs_current = []
dfs_next = []
dfs_outlook = []

for file in latest_file_by_date.values():
    result = process_corn(file)
    if result is None:
        continue

    df_corn, df_current, df_next, df_outlook = result
    dfs_corn.append(df_corn)
    dfs_current.append(df_current)
    dfs_next.append(df_next)
    dfs_outlook.append(df_outlook)

df_corn = pd.concat(dfs_corn, ignore_index=True)
df_corn_current = pd.concat(dfs_current, ignore_index=True, sort=False)
df_corn_next = pd.concat(dfs_next, ignore_index=True, sort=False)
df_corn_outlook = pd.concat(dfs_outlook, ignore_index=True, sort=False)

df_corn_current = df_corn_current.reset_index(drop=True)
df_corn_next = df_corn_next.reset_index(drop=True)
df_corn_outlook = df_corn_outlook.reset_index(drop=True)
df_corn = df_corn.reset_index(drop=True)

df_corn_current.to_excel(CORN_CURRENT)
df_corn_next.to_excel(CORN_NEXT)
df_corn_outlook.to_excel(CORN_OUTLOOK)
df_corn.to_excel(CORN)

print(f"✅ df_corn created with {len(df_corn)} lines.")
print(f"✅ df_corn_current created with {len(df_corn_current)} lines.")
print(f"✅ df_corn_next created with {len(df_corn_next)} lines.")
print(f"✅ df_corn_outlook created with {len(df_corn_outlook)} lines.")


⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-09_wasde-05-09-2001_rice-revision.xls: Worksheet named 'Page 22' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-10_wasde-05-10-2001_wheat-revision.xls: Worksheet named 'Page 22' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-05-10_wasde-05-10-2002_world_grains_revision.xls: Worksheet named 'Page 22' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-11-12_wasde-11-12-2002_China_wheat_revision.xls: Worksheet named 'Page 22' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2003-07-11_wasde-07-11-2003_world_oilseed_revision.xls: Worksheet named 'Page 22' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2004-05-12_wasde-05-12-2004_world_coarse_grains_revision.xls: 

5. Processing soybean

In [5]:
files = list(WASDE_FOLDER.glob('*.xls'))

latest_file_by_date = {}
for file in files:
    basename = os.path.basename(file)
    report_date = basename.split('_')[0]
    latest_file_by_date[report_date] = file

dfs_soybean = []
dfs_current = []
dfs_next = []
dfs_outlook = []

for file in latest_file_by_date.values():
    result = process_soybean(file)
    if result is None:
        continue

    df_soybean, df_current, df_next, df_outlook = result
    dfs_soybean.append(df_soybean)
    dfs_current.append(df_current)
    dfs_next.append(df_next)
    dfs_outlook.append(df_outlook)

df_soybean = pd.concat(dfs_soybean, ignore_index=True)
df_soybean_current = pd.concat(dfs_current, ignore_index=True, sort=False)
df_soybean_next = pd.concat(dfs_next, ignore_index=True, sort=False)
df_soybean_outlook = pd.concat(dfs_outlook, ignore_index=True, sort=False)

df_soybean_current = df_soybean_current.reset_index(drop=True)
df_soybean_next = df_soybean_next.reset_index(drop=True)
df_soybean_outlook = df_soybean_outlook.reset_index(drop=True)
df_soybean = df_soybean.reset_index(drop=True)

df_soybean_current.to_excel(SOYBEAN_CURRENT)
df_soybean_next.to_excel(SOYBEAN_NEXT)
df_soybean_outlook.to_excel(SOYBEAN_OUTLOOK)
df_soybean.to_excel(SOYBEAN)

print(f"✅ df_soybean created with {len(df_soybean)} lines.")
print(f"✅ df_soybean_current created with {len(df_soybean_current)} lines.")
print(f"✅ df_soybean_next created with {len(df_soybean_next)} lines.")
print(f"✅ df_soybean_outlook created with {len(df_soybean_outlook)} lines.")


⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-09_wasde-05-09-2001_rice-revision.xls: Worksheet named 'Page 28' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-10_wasde-05-10-2001_wheat-revision.xls: Worksheet named 'Page 28' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-05-10_wasde-05-10-2002_world_grains_revision.xls: Worksheet named 'Page 28' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-11-12_wasde-11-12-2002_China_wheat_revision.xls: Worksheet named 'Page 28' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2003-07-11_wasde-07-11-2003_world_oilseed_revision.xls: Worksheet named 'Page 28' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2004-05-12_wasde-05-12-2004_world_coarse_grains_revision.xls: 

6. Processing soybean oil

In [6]:
files = list(WASDE_FOLDER.glob('*.xls'))

latest_file_by_date = {}
for file in files:
    basename = os.path.basename(file)
    report_date = basename.split('_')[0]
    latest_file_by_date[report_date] = file

dfs_soybean_oil = []
dfs_current = []
dfs_next = []
dfs_outlook = []

for file in latest_file_by_date.values():
    result = process_soybean_oil(file)
    if result is None:
        continue

    df_soybean_oil, df_current, df_next, df_outlook = result
    dfs_soybean_oil.append(df_soybean_oil)
    dfs_current.append(df_current)
    dfs_next.append(df_next)
    dfs_outlook.append(df_outlook)

df_soybean_oil = pd.concat(dfs_soybean_oil, ignore_index=True)
df_soybean_oil_current = pd.concat(dfs_current, ignore_index=True, sort=False)
df_soybean_oil_next = pd.concat(dfs_next, ignore_index=True, sort=False)
df_soybean_oil_outlook = pd.concat(dfs_outlook, ignore_index=True, sort=False)

df_soybean_oil_current = df_soybean_oil_current.reset_index(drop=True)
df_soybean_oil_next = df_soybean_oil_next.reset_index(drop=True)
df_soybean_oil_outlook = df_soybean_oil_outlook.reset_index(drop=True)
df_soybean_oil = df_soybean_oil.reset_index(drop=True)

df_soybean_oil_current.to_excel(SOYBEAN_OIL_CURRENT)
df_soybean_oil_next.to_excel(SOYBEAN_OIL_NEXT)
df_soybean_oil_outlook.to_excel(SOYBEAN_OIL_OUTLOOK)
df_soybean_oil.to_excel(SOYBEAN_OIL)

# Logs
print(f"✅ df_soybean_oil created with {len(df_soybean_oil)} lines.")
print(f"✅ df_soybean_oil_current created with {len(df_soybean_oil_current)} lines.")
print(f"✅ df_soybean_oil_next created with {len(df_soybean_oil_next)} lines.")
print(f"✅ df_soybean_oil_outlook created with {len(df_soybean_oil_outlook)} lines.")


⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-09_wasde-05-09-2001_rice-revision.xls: Worksheet named 'Page 30' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-10_wasde-05-10-2001_wheat-revision.xls: Worksheet named 'Page 30' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-05-10_wasde-05-10-2002_world_grains_revision.xls: Worksheet named 'Page 30' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-11-12_wasde-11-12-2002_China_wheat_revision.xls: Worksheet named 'Page 30' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2003-07-11_wasde-07-11-2003_world_oilseed_revision.xls: Worksheet named 'Page 30' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2004-05-12_wasde-05-12-2004_world_coarse_grains_revision.xls: 

7. Processing soybean meal

In [7]:
files = list(WASDE_FOLDER.glob('*.xls'))

latest_file_by_date = {}
for file in files:
    basename = os.path.basename(file)
    report_date = basename.split('_')[0]
    latest_file_by_date[report_date] = file  


dfs_soybean_meal = []
dfs_current = []
dfs_next = []
dfs_outlook = []

for file in latest_file_by_date.values():
    result = process_soybean_meal(file)
    if result is None:
        continue

    df_soybean_meal, df_current, df_next, df_outlook = result
    dfs_soybean_meal.append(df_soybean_meal)
    dfs_current.append(df_current)
    dfs_next.append(df_next)
    dfs_outlook.append(df_outlook)

df_soybean_meal = pd.concat(dfs_soybean_meal, ignore_index=True)
df_soybean_meal_current = pd.concat(dfs_current, ignore_index=True, sort=False)
df_soybean_meal_next = pd.concat(dfs_next, ignore_index=True, sort=False)
df_soybean_meal_outlook = pd.concat(dfs_outlook, ignore_index=True, sort=False)

df_soybean_meal_current = df_soybean_meal_current.reset_index(drop=True)
df_soybean_meal_next = df_soybean_meal_next.reset_index(drop=True)
df_soybean_meal_outlook = df_soybean_meal_outlook.reset_index(drop=True)
df_soybean_meal = df_soybean_meal.reset_index(drop=True)

df_soybean_meal_current.to_excel(SOYBEAN_MEAL_CURRENT)
df_soybean_meal_next.to_excel(SOYBEAN_MEAL_NEXT)
df_soybean_meal_outlook.to_excel(SOYBEAN_MEAL_OUTLOOK)
df_soybean_meal.to_excel(SOYBEAN_MEAL)

# Logs
print(f"✅ df_soybean_meal created with {len(df_soybean_meal)} lines.")
print(f"✅ df_soybean_meal_current created with {len(df_soybean_meal_current)} lines.")
print(f"✅ df_soybean_meal_next created with {len(df_soybean_meal_next)} lines.")
print(f"✅ df_soybean_meal_outlook created with {len(df_soybean_meal_outlook)} lines.")


⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-09_wasde-05-09-2001_rice-revision.xls: Worksheet named 'Page 29' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2001-05-10_wasde-05-10-2001_wheat-revision.xls: Worksheet named 'Page 29' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-05-10_wasde-05-10-2002_world_grains_revision.xls: Worksheet named 'Page 29' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2002-11-12_wasde-11-12-2002_China_wheat_revision.xls: Worksheet named 'Page 29' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2003-07-11_wasde-07-11-2003_world_oilseed_revision.xls: Worksheet named 'Page 29' not found
⚠️ Error reading C:\Users\d1eg0\Desktop\python\Prediction\sarimax\data\wasde_files\2004-05-12_wasde-05-12-2004_world_coarse_grains_revision.xls: 

8. Concat of commodity data

In [8]:
df_commodity = pd.concat([df_wheat, df_corn, df_soybean, df_soybean_oil, df_soybean_meal], ignore_index=True)
df_commodity.to_excel(COMMODITY)